In [6]:
pip install "gradio-4.27.0.tar.gz"

Processing ./gradio-4.27.0.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 

In [2]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.5 MB/s eta 0:00:00


In [4]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-6hydki_i
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-6hydki_i
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=3fa155a13bdb896b2c641a1b48a08ed9bbde1dec5bbabd94bba81b6e6a0970d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-q54cwidn/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [5]:
pip install opencv-python pycocotools matplotlib onnxruntime onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.2 MB/s eta 0:00:00


In [63]:
import gradio as gr
import numpy as np
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry
device = "cuda"
sam_checkpoint = "drive/MyDrive/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type] (checkpoint=sam_checkpoint)
sam. to(device)
predictor = SamPredictor (sam)
pipe = StableDiffusionInpaintPipeline.from_pretrained(
  "stabilityai/stable-diffusion-2-inpainting",
   torch_dtype=torch.float16,
)

pipe = pipe. to(device)

selected_pixels=[]
with gr. Blocks() as demo:
  with gr.Row():
    input_img = gr. Image(label="Input")
    mask_img = gr. Image (label="Mask")
    output_img = gr. Image(label="Output")

  with gr.Row():
    prompt_text=gr.Textbox(lines=1,label="Prompt")

  with gr.Row():
    submit= gr.Button("Submit")

  def generate_mask(image,evt:gr.SelectData):
    selected_pixels.append(evt.index)

    predictor.set_image(image)
    input_points = np.array(selected_pixels)
    input_labels=np.ones(input_points.shape[0])
    mask,_,_=predictor.predict(
        point_coords=input_points,
        point_labels=input_labels,
        multimask_output=False
    )

    #(n,sz,sz)
    mask=np.logical_not(mask)
    mask=Image.fromarray(mask[0,:,:])
    return mask

  def inpaint(image,mask,prompt):
    image=Image.fromarray(image)
    mask=Image.fromarray(mask)

    image=image.resize((512,512))
    mask=mask.resize((512,512))

    output = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask).images[0]

    return output

  input_img.select(generate_mask,[input_img],[mask_img])
  submit.click(
      inpaint,
      inputs=[input_img,mask_img,prompt_text],
      outputs=[output_img],
      )
if __name__=="__main__":
    demo.launch()



Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f833c190845167ea75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
